In [1]:
import smartsheet_dataframe as sms
import pandas as pd
import numpy as np
token = #your access token
lcm_ind1_id = 8801004925439876 #sheet1 IND-LAB data
lcm_ind0_id = 1339307588904836 #historical data (pre 2023 data) IND-LAB data
lcm_bn1_id = 1540237486411652 #sheet1 HARD-BN data
lcm_bn2_id = 7164483025719172 #sheet2 HARD-BN data

In [2]:
def fix_percentage_bool(column):
    column = column.astype(str)  
    column = column.str.replace('%', '')  
    column = column.astype(float) / 100  
    column = column.apply(lambda x: 1 if 0 < x < 1 else x)  
    return column

def fix_percentage(column):
    column = column.astype(str)  
    column = column.str.replace('%', '')  
    column = column.astype(float) / 100    
    return column

In [3]:
#IND-LAB EXTRACTION
lcm_ind1 = sms.get_sheet_as_df(token=token, sheet_id=lcm_ind1_id)
lcm_ind1 = lcm_ind1.astype(str)
lcm_ind0 = sms.get_sheet_as_df(token=token, sheet_id=lcm_ind0_id)
lcm_ind0 = lcm_ind0.astype(str)

In [4]:
#IND-LAB COLUMN HOMOLOGATION & APPEND
lcm_ind0 = lcm_ind0.rename(columns={'#Embryos extracted': '#Total Embryos', '#Diploids': '#Diploids Embryos', '#Haploids': '#Haploids Embryos'})
lcm_ind0 = lcm_ind0.drop(['row_id', 'parent_id', 'id','Petri 1', '#Pollin. Plants', '%Pollin. Plants', '#Surviv. Plants', 'Embryo Size', 'ER Method'], axis=1)
lcm_ind1 = lcm_ind1.drop(['row_id', 'parent_id'], axis=1)
merged_ind = pd.concat([lcm_ind0, lcm_ind1], ignore_index=True)
#merged_ind.describe()

In [5]:
#IND-LAB DATA FORMAT AND CLEANING
merged_ind['IND-ID'] = merged_ind['IND-ID'].str.replace('.0', '', regex=False).astype(str)
merged_ind['Plant ID'] = pd.to_numeric(merged_ind['Plant ID'], errors='coerce')
merged_ind = merged_ind[pd.notnull(merged_ind['Plant ID'])]
merged_ind.replace('', np.nan, inplace=True)
ind_int_columns = ['Shoot (DAS)', 'Silk Prep (DAS)', 'Pollination (DAS)', 'Silk to Poll (Days)', 'Harvest (DAS)', '#Total Embryos', '#Diploids Embryos', '#Haploids Embryos', '#Dead Embryos', '#Embryos in Petri', 'Embryos Wavin', 'Harvest to proces (days)', 'Total selected seedlings', 'Category 3 seedlings', 'Category 2 seedlings', 'Category 1 seedlings']
ind_float_columns = ['%Haploidy', 'Stalk diameter (cm)', 'Ear height (cm)', 'Ear lenght (cm)']

merged_ind[ind_int_columns] = merged_ind[ind_int_columns].astype(float)
merged_ind[ind_float_columns] = merged_ind[ind_float_columns].astype(float)

merged_ind['%Germin.'] = fix_percentage_bool(merged_ind['%Germin.'])
merged_ind['%Pollination'] = fix_percentage_bool(merged_ind['%Pollination'])
merged_ind['Germination Test 1 (0m)'] = fix_percentage(merged_ind['Germination Test 1 (0m)'])
merged_ind['Germination Test 2 (3m)'] = fix_percentage(merged_ind['Germination Test 2 (3m)'])
merged_ind['Germination Test 3 (6m)'] = fix_percentage(merged_ind['Germination Test 3 (6m)'])
#merged_ind.info()

In [6]:
#HARD-BN EXTRACTION
lcm_bn1 = sms.get_sheet_as_df(token=token, sheet_id=lcm_bn1_id)
lcm_bn1 = lcm_bn1.astype(str)
lcm_bn2 = sms.get_sheet_as_df(token=token, sheet_id=lcm_bn2_id)
lcm_bn2 = lcm_bn2.astype(str)

In [7]:
#HARD-BN COLUMN HOMOLOGATION & APPEND
merged_bn = pd.concat([lcm_bn1, lcm_bn2], ignore_index=True)
merged_bn = merged_bn.drop(['row_id', 'parent_id'], axis=1)

In [8]:
#HARD-BN DATA FORMAT AND CLEANING
merged_bn.replace('', np.nan, inplace=True)
merged_bn['IND-ID'] = merged_bn['IND-ID'].str.replace('.0', '', regex=False).astype(str)
merged_bn = merged_bn.drop(['Induction MATID', 'Site','Planting number','Planting time','Planting size','GH Type','GH number','Plant ID','Region','is parent'],axis=1)
bn_float_columns = ['Selection to Potting (days)', 'Potted plants', 'Potting to Trasplant (days)', 'Field trasplant count', 'N° of plant germinated', 'Survival rate', 'Trasplant to Pollination (days)', 'Nursery pollination count', 'Nursery stem diameter (cm)', 'Nursery ear height (cm)', 'Nursery harvest count', 'Pollination to Harvest (days)', 'Nursery ear length (cm)', 'Kernels by ear']
merged_bn[bn_float_columns] = merged_bn[bn_float_columns].astype(float)
#merged_bn.info()

In [9]:
#FULL LCM DATA MERGING & EXPORT
full_lcm_data = pd.merge(merged_ind, merged_bn, on='IND-ID', how='left')
filepath = 'merged_ind.csv'
merged_ind.to_csv(filepath, index=False)
print(f"Merged Induction Data successfully merged and exported to {filepath}")
filepath = 'merged_bn.csv'
merged_bn.to_csv(filepath, index=False)
print(f"Merged BN Data successfully merged and exported to {filepath}")
filepath = 'full_lcm_data.csv'
full_lcm_data.to_csv(filepath, index=False)
print(f"Full LCM Data successfully merged and exported to {filepath}")

Merged Induction Data successfully merged and exported to merged_ind.csv
Merged BN Data successfully merged and exported to merged_bn.csv
Full LCM Data successfully merged and exported to full_lcm_data.csv
